In [1]:
import gf as primpoly
import numpy as np
from copy import deepcopy

In [2]:
class GaloisField:
        
    def add(self,x, y):
        return x ^ y

    def sub(self,x, y):
        # in binary galois field, subtraction is just the same as addition (since we mod 2)
        return x ^ y 
    
    def mul(self,x, y):
        if x==0 or y==0:
            return 0
        return self.exp[(self.log[x] + self.log[y]) % (self.length-1)]

    def div(self,x,y):
        if y==0:
            raise ZeroDivisionError()
        if x==0:
            return 0
        return self.exp[(self.log[x] + (self.length-1) - self.log[y]) % (self.length-1)]

    def gf_pow(self, x, power):
        return self.exp[(self.log[x] * power) % (self.length-1)]
    
    def gf_inverse(self, x):
        return self.exp[(self.length-1) - self.log[x]]
    
    def conv(self, a, b):
        if (len(b) > len(a)):
            temp = a
            a = b
            b = temp

        a_pad = [*([0]*(len(b)-1)), *a, *([0]*(len(b)-1))]
        b_rev = list(reversed(b))

        res = [0]*(len(a_pad)-1)

        for i in range(0,len(a_pad)-len(b)+1):
            partial_sum = 0
            for j in range(0,len(b)):
                partial_sum = self.add(partial_sum,self.mul(a_pad[i+j],b[j]))
            res[i] = partial_sum

        return res
    
    def init_tables(self):
        
        '''Precompute the logarithm and anti-log tables for faster computation later, 
           using the provided primitive polynomial.'''
        # prim is the primitive (binary) polynomial. Since it's a polynomial in the binary sense,
        # it's only in fact a single galois field value between 0 and 255, and not a list of gf values.
        
        m = self.order
        prim = self.prim
        gf_length = self.length
        gf_exp = [0] * gf_length # anti-log (exponential) table
        gf_log = [0] * gf_length # log table
        # For each possible value in the galois field 2^8, 
        # we will pre-compute the logarithm and anti-logarithm (exponential) of this value
        x = 1
        for i in range(0, gf_length-1):
            gf_exp[i] = x # compute anti-log for this value and store it in a table
            gf_log[x] = i # compute log at the same time

            x <<= 1 # multiply by 2 (change 1 by another number y to multiply by a power of 2^y)
            if x >= gf_length:
                # substract the primary polynomial to the current value 
                # (instead of Galois Field length, so that we get a unique set made of coprime numbers), 
                # this is the core of the tables generation
                x ^= prim 
                
        return [gf_log, gf_exp]
    
    def det(self, matrix, mul = 1):
        '''Recursively computes the determinant of a square matrix in Galois Field'''
        
        width = len(matrix)
        if width == 1:
            return self.mul(mul, matrix[0][0])
        else:
            total = 0
            for i in range(width):
                m = []
                for j in range(1, width):
                    buff = []
                    for k in range(width):
                        if k != i:
                            buff.append(matrix[j][k])
                    m.append(buff)
                total = self.add(total, self.mul(mul, self.det(m, matrix[0][i])))
            return total
    
    def solveLinearSystem(self, matrix, array):
        '''Solves a linear system in Galois Field using Cramer's rule and 
           returns the answer to array ans[]''' 
        
        ans = []
        for i in range(0, len(matrix)):
            
            # We need to use deepcopy because temp = matrix returns a pointer to matrix
            # and all changes to temp also affect matrix
            temp = deepcopy(matrix)
            
            # We iteratively substitute each column of the matrix with the array
            # and the compute its determinant in the Galois Field
            for j in range(0, len(array)):
                temp[j][i] = array[j]

            ans.append(self.div(S.det(temp), self.det(matrix)))
        return ans
    
                
    def __init__(self, x, m, d):
        '''Class initialization method'''
        
        self.element = x
        self.order = m
        self.prim = d
        self.length = 2**m
        [self.log, self.exp] = self.init_tables()
        

# Reed Solomon Decoding 

First, the original all-zero bitstream $u(x) = u_0 + u_1x + \cdots + u_{n-1}x^{n-1}$ is generated and it gets corrupted with a random error $e(x) = e_0 + e_1x + \cdots + e_{n-1}x^{n-1}$. The received bitstream is split in symbols of m bits to create the $r(x) = u(x) + e(x) = r_0 + r_1x + \cdots + r_{n-1}x^{n-1}$ polynomial. The syndrom $S$ is then calculated using the formula $S_i = r(\alpha^i)$. We assume that there are $\nu$ errors at positions $j_1, j_2, \cdots, j_{\nu}$ of the polynomial $e(x)$. Let $\beta_i = \alpha^{j_i}$ and t the number of erroneous symbols that can be corrected, so that the syndroms are calculated as:

$\begin{array}
\text S_1 = r(\alpha) = e_{j_1}\beta_1 + e_{j_2}\beta_2 + \cdots + e_{j_{\nu}}\beta_{\nu} \\
\text S_2 = r(\alpha^2) = e_{j_1}{\beta_1}^2 + e_{j_2}{\beta_2}^2 + \cdots + e_{j_{\nu}}{\beta_{\nu}}^2 \\
\vdots \\
\text S_{2t} = r(\alpha^{2t}) = e_{j_1}{\beta_1}^{2t} + e_{j_2}{\beta_2}^{2t} + \cdots + e_{j_{\nu}}{\beta_{\nu}}^{2t} \\
\end{array}$

The polynomial $\sigma(x)$ is computed using an autoregressive model. We make a matrix from the syndroms, where the first t syndroms are used to predict the next syndrome. The matrix is contructed as shown below:

$\begin{bmatrix} 
S_1 & S_2 & \cdots & S_{t-1} & S_t \\
S_2 & S_3 & \cdots & S_t & S_{t+1} \\
&& \vdots \\
S_{t-1} & S_t & \cdots & S_{2t-3} & S_{2t-2} \\
S_t & S_{t+1} & \cdots & S_{2t-2} & S_{2t-1} \\
\end{bmatrix} 
 * 
\begin{bmatrix}
\sigma_t\\
\sigma_{t-1}\\
\vdots\\
\sigma_2\\
\sigma_1\\
\end{bmatrix}
 = 
\begin{bmatrix}
-S_{t+1}\\
-S_{t+2}\\
\vdots\\
-S_{2t-1}\\
-S_{2t}\\
\end{bmatrix}$

Then the value of the errors $e_{j_1}$ can be calculated by solving any t of the equatations written above:

$\begin{bmatrix} 
\beta_1 & \beta_2 & \cdots & \beta_{\nu-1} & \beta_{\nu} \\
{\beta_1}^2 & {\beta_2}^2 & \cdots & {\beta_{\nu-1}}^2 & {\beta_{\nu}}^2 \\
&& \vdots \\
{\beta_1}^t & {\beta_2}^t & \cdots & {\beta_{\nu-1}}^t & {\beta_{\nu}}^t \\
\end{bmatrix} 
 * 
\begin{bmatrix}
e_{j_1}\\
e_{j_2}\\
\vdots\\
e_{j_t}\\
\end{bmatrix}
 = 
\begin{bmatrix}
S_1\\
S_2\\
\vdots\\
S_t\\
\end{bmatrix}$

Finally, the original message can be retreived by adding the errors to the received signal

In [3]:
t = 3 # maximum number of erroneous symbols that can be corrected
m = 4 # order of the Galois Field
n = 2**m - 1
d = primpoly.find_prime_polys(2,m,single=True)

print("Prime polynomial of GF(2^{}) in binary form: {}".format(m, bin(d)))

# The first non-zero, non-one element of the Galois Field
a = GaloisField(2,m,d)

Prime polynomial of GF(2^4) in binary form: 0b10011


In [4]:
# Generate an all-zero codeword of length n*m bits and deliberate corrupt some bits, 
# whose locations are specified in e_pos array

rb = np.zeros(m*n, dtype = int)

# Damage a number of random bits
err_num = 3  # Caution: If more than t symbols get damaged, Reed Solomon decoding fails 
e_pos = np.random.randint(0, n*m, err_num, dtype=int)
print("Position of the corrupted bits: ", e_pos)

# Else corrupt specific bits (for debugging purposes)
# e_pos = np.array([12, 13, 15, 27, 48, 49])
# e_pos = np.array([12,13,14,25,26,52,54])

for i in range(0,len(e_pos)):
    rb[e_pos[i]] = not(rb[e_pos[i]])

# Split the message in symbols of length m bits
rm = np.zeros(len(rb)//m, dtype=int)
for i in range(0,len(rb)//m):
    for j in range (0,m):
        rm[i] = (rm[i] << 1) + rb[i*m+j]

print("The corrupted message symbols in decimal form are: ",rm)
r = GaloisField(rm,m,d)

Position of the corrupted bits:  [43 35  1]
The corrupted message symbols in decimal form are:  [4 0 0 0 0 0 0 0 1 0 1 0 0 0 0]


In [5]:
# Calculate syndroms in array s
# s[i] = r(a^i) for i = 1...2t where r(x) the currupted message in polynomial form

s = GaloisField([],m,d)
for i in range(0,2*t):
    s.element.append(r.element[n-1])
    for j in range(0, n-1):
        temp = GaloisField(2,m,d)
        temp.element = a.gf_pow(a.element,i+1)
        temp.element = temp.gf_pow(temp.element, j+1)
        s.element[i] = s.add(s.element[i],temp.mul(temp.element, r.element[n-j-2]))
    
print("The syndrom of the message is:", s.element)

The syndrom of the message is: [13, 11, 14, 5, 8, 8]


In [6]:
# Compute sigma polynomial using auto-regression method
# by solving the linear system S*sigma = cons_s

S = GaloisField(np.zeros([t, t], dtype=int),m,d)
cons_s = GaloisField(np.zeros(t, dtype=int),m,d)
for i in range(0,t):
    for j in range(0,t):
        S.element[i][j] = s.element[i+j]
    cons_s.element[i] = s.element[t+i]

if (S.det(S.element) == 0):
    print("The syndrom cannot be defined, because det(S) == 0")
else:
    sigma = S.solveLinearSystem(S.element, cons_s.element)

    print("S\t\t sigma\t cons_s")
    for i in range(0, len(S.element)):
        print(S.element[i][:], "\t", sigma[i], "\t", cons_s.element[i])

S		 sigma	 cons_s
[13 11 14] 	 10 	 5
[11 14  5] 	 9 	 8
[14  5  8] 	 6 	 8


In [7]:
# Find roots of sigma polynomial
s_roots_exp = []
for i in range(0, n-1):
    # Compute the value of polynomial sigma where x -> a^i for i = 0...n-1
    ai = a.gf_pow(a.element, i+1)
    sr = GaloisField(1, m, d)
    
    # Construct polynomial from coefficients stored in array sigma[]
    # (the highest order coefficient is stored in sigma[0])
    for j in range(0, t):
        temp = a.mul(sigma[j],a.gf_pow(ai,t-j))
        sr.element = sr.add(sr.element,temp)
    
    if (sr.element == 0):
        # s_roots_exp contains the powers of a for which sigma(a^i) == 0
        s_roots_exp.append(i+1)
        
print(s_roots_exp)

[1, 9, 11]


In [8]:
# The error positions can be found by inversing the roots of sigma polynomial

b = []
err_pos = []
for i in range(0, len(s_roots_exp)):
    root = a.gf_pow(a.element, s_roots_exp[i])
    b.append(a.gf_inverse(root))
    
    # Find error positions form most significant bit
    err_pos.append(n - a.log[b[i]])
    
print("Symbols with errors are: ", err_pos)

Symbols with errors are:  [1, 9, 11]


In [9]:
# Having found the erroneous symbols, we need to determine the value of each error
# We solve the linear system B = err * cons_b

B = GaloisField([],m,d)
cons_b = GaloisField([],m,d)
for i in range(0,t):
    
    for j in range(0,t):
        B.element.append(B.gf_pow(b[j],i+1))

    cons_b.element.append(s.element[i])

B.element = np.reshape(B.element, [t, len(B.element)//t])
err = B.solveLinearSystem(B.element, cons_b.element)

print("B\t\t err\t cons_b")
for i in range(0, len(S.element)):
    print(B.element[i][:], "\t", err[i], "\t", cons_b.element[i])

B		 err	 cons_b
[ 9 12  3] 	 4 	 13
[13 15  5] 	 1 	 11
[15  8 15] 	 1 	 14


In [10]:
# The corrupted can be fixed by subtracting the error values of err array
# from the erroneous symbols as specified in err_pos array

corrected = GaloisField([], m, d)
corrected.element = deepcopy(r.element)
for i in range(0, len(err)):
    corrected.element[err_pos[i]-1] = corrected.add(r.element[err_pos[i]-1], err[i])
    
print("Corrupted message:", r.element)
print("Corrected message:", corrected.element)

Corrupted message: [4 0 0 0 0 0 0 0 1 0 1 0 0 0 0]
Corrected message: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
